# Learn Posture

use machine learning to recognize robot's posture (following the example in [scikit-learn-intro.ipynb](./scikit-learn-intro.ipynb) )

## 1. Data collection

We have colleceted data before, you need to add new data if you want to add new posture.

* the dateset are in *robot_pose_data* folder
* each file contains the data belongs to this posture, e.g. the data in *Back* file are collected when robot was in "Back" posture
* the data file can be load by ```pickle```, e.g. ```pickle.load(open('Back'))```, the data is a list of feature data
* the features (e.g. each row of the data) are ['LHipYawPitch', 'LHipRoll', 'LHipPitch', 'LKneePitch', 'RHipYawPitch', 'RHipRoll', 'RHipPitch', 'RKneePitch', 'AngleX', 'AngleY'], where 'AngleX' and 'AngleY' are body angle (e.g. ```Perception.imu```) and others are joint angles.

## 2. Data preprocessing

In [15]:
%pylab inline
import pickle
from os import listdir, path
import numpy as np
from sklearn import svm, metrics

ROBOT_POSE_DATA_DIR = 'robot_pose_data'

Populating the interactive namespace from numpy and matplotlib


In [16]:
classes = sorted(listdir(ROBOT_POSE_DATA_DIR))
print classes

['Back', 'Belly', 'Crouch', 'Frog', 'HeadBack', 'Knee', 'Left', 'Right', 'Sit', 'Stand', 'StandInit']


In [17]:
def load_pose_data(i):
    '''load pose data from file'''
    data = []
    target = []
    # YOUR CODE HERE
    
    filename = path.join(ROBOT_POSE_DATA_DIR, classes[i])
    data = pickle.load(open(filename))
    target = [i] * len(data)
    return data, target

In [18]:
# load all the data
all_data = []
all_target = []
for i in range(len(classes)):
    data, target = load_pose_data(i)
    all_data.extend(data)
    all_target.extend(target)

print 'total number of data', len(all_data), 'of targets', len(all_target)

total number of data 224 of targets 224


In [19]:
# shuffule data
permutation = np.random.permutation(len(all_data))
n_training_data = int(len(all_data) * 0.7)
training_data = permutation[:n_training_data]

## 3. Learn on training data

In scikit-learn, an estimator for classification is a Python object that implements the methods fit(X, y) and predict(T). An example of an estimator is the class sklearn.svm.SVC that implements support vector classification.

In [20]:
clf = svm.SVC(gamma=0.001, C=100.)

### learning

In [21]:
clf.fit(np.asarray(all_data)[training_data], np.asarray(all_target)[training_data])

SVC(C=100.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

### predicting

In [22]:
clf.predict(all_data[-1:]), all_target[-1:]

(array([10]), [10])

In [23]:
def evaluate(expected, predicted):
    print("Classification report:\n%s\n" % metrics.classification_report(expected, predicted))

    print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

In [24]:
expected = np.asarray(all_target)[training_data]
predicted = clf.predict(np.asarray(all_data)[training_data])

evaluate(expected, predicted)

Classification report:
             precision    recall  f1-score   support

          0       1.00      1.00      1.00        20
          1       1.00      1.00      1.00        15
          2       1.00      1.00      1.00        22
          3       1.00      1.00      1.00         9
          4       1.00      1.00      1.00         7
          5       1.00      1.00      1.00         7
          6       1.00      1.00      1.00        13
          7       1.00      1.00      1.00         7
          8       1.00      1.00      1.00        15
          9       1.00      1.00      1.00         9
         10       1.00      1.00      1.00        32

avg / total       1.00      1.00      1.00       156


Confusion matrix:
[[20  0  0  0  0  0  0  0  0  0  0]
 [ 0 15  0  0  0  0  0  0  0  0  0]
 [ 0  0 22  0  0  0  0  0  0  0  0]
 [ 0  0  0  9  0  0  0  0  0  0  0]
 [ 0  0  0  0  7  0  0  0  0  0  0]
 [ 0  0  0  0  0  7  0  0  0  0  0]
 [ 0  0  0  0  0  0 13  0  0  0  0]
 [ 0  0  0  0 

## 4. Evaluate on the test data

In [25]:
expected = []
predicted = []
test_data = permutation[n_training_data:]
expected = np.asarray(all_target)[test_data]
predicted = clf.predict(np.asarray(all_data)[test_data])

evaluate(expected, predicted)

Classification report:
             precision    recall  f1-score   support

          0       1.00      1.00      1.00         5
          1       1.00      1.00      1.00         4
          2       0.89      1.00      0.94         8
          3       1.00      1.00      1.00         1
          4       1.00      1.00      1.00         3
          5       1.00      1.00      1.00         3
          6       1.00      1.00      1.00         7
          7       1.00      0.75      0.86         4
          8       1.00      1.00      1.00        11
          9       1.00      1.00      1.00         2
         10       1.00      1.00      1.00        20

avg / total       0.99      0.99      0.98        68


Confusion matrix:
[[ 5  0  0  0  0  0  0  0  0  0  0]
 [ 0  4  0  0  0  0  0  0  0  0  0]
 [ 0  0  8  0  0  0  0  0  0  0  0]
 [ 0  0  0  1  0  0  0  0  0  0  0]
 [ 0  0  0  0  3  0  0  0  0  0  0]
 [ 0  0  0  0  0  3  0  0  0  0  0]
 [ 0  0  0  0  0  0  7  0  0  0  0]
 [ 0  0  1  0 

## 5. Deploy to the real system

We can simple use `pickle` module to serialize the trained classifier.

In [26]:
import pickle
ROBOT_POSE_CLF = 'robot_pose.pkl'
pickle.dump(clf, open(ROBOT_POSE_CLF, 'w'))

Then, in the application we can load the trained classifier again.

In [27]:
clf2 = pickle.load(open(ROBOT_POSE_CLF))
clf2.predict(all_data[-1:]), all_target[-1:]

(array([10]), [10])